In [ ]:
from google.colab import drive
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Step 1: Mount Google Drive to access data
drive.mount('/content/drive')

# Define directories for training and test data
train_dir = '/content/drive/My Drive/IISC work/ML Project/Data/Train_Data'
test_dir = '/content/drive/My Drive/IISC work/ML Project/Data/Test_Data'

# Step 2: Load and Preprocess Training Data
IMG_SIZE = (128, 128)
ALLOWED_EXTENSIONS = {'.jpg', '.jpeg', '.png'}

# Function to load training data
def load_data(train_dir):
    images = []
    labels = []
    label_map = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'S': 5}

    for folder in os.listdir(train_dir):
        folder_path = os.path.join(train_dir, folder)
        if os.path.isdir(folder_path) and folder in label_map:
            for img_name in os.listdir(folder_path):
                if not any(img_name.lower().endswith(ext) for ext in ALLOWED_EXTENSIONS):
                    continue
                img_path = os.path.join(folder_path, img_name)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, IMG_SIZE)
                    images.append(img)
                    labels.append(label_map[folder])
    return np.array(images), np.array(labels)

# Load training data
if os.path.exists(train_dir):
    images, labels = load_data(train_dir)
    print(f"Loaded {len(images)} images.")
    print(f"Shape of images array: {images.shape}")
    print(f"Shape of labels array: {labels.shape}")

# Step 3: Prepare Data for Training
labels = labels - 1  # Adjust labels to be 0-based

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Advanced Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=30,  # Reduce rotation range
    width_shift_range=0.2,  # Reduce shift range
    height_shift_range=0.2,  # Reduce shift range
    shear_range=0.2,  # Reduce shear range
    zoom_range=0.2,  # Reduce zoom
    horizontal_flip=True,
    fill_mode='nearest',
    rescale=1./255
)
val_datagen = ImageDataGenerator(rescale=1./255)

# Step 4: Define the Model Using EfficientNetB0
base_model = EfficientNetB0(input_shape=(128, 128, 3), include_top=False, weights='imagenet')

# Unfreeze fewer layers for fine-tuning
for layer in base_model.layers[-20:]:  # Unfreeze last 20 layers for fine-tuning
    layer.trainable = True

# Define model architecture
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),  # Reduce neurons in dense layer
    Dropout(0.3),  # Reduce dropout rate to retain more information
    Dense(256, activation='relu'),  # Additional dense layer
    Dropout(0.3),
    Dense(5, activation='softmax')  # Assuming 5 classes
])

# Compile the model with AdamW optimizer
model.compile(optimizer=AdamW(learning_rate=0.00005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Step 5: Training the Model
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00005)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Compute class weights (optional: you can experiment with and without this)
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = {i: weight for i, weight in enumerate(class_weights)}

batch_size = 32  # Use a smaller batch size

# Train the model with data augmentation, class weights, and callbacks
model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
          validation_data=val_datagen.flow(X_val, y_val),
          epochs=40,  # Increase epochs for better convergence
          class_weight=class_weights,  # Experiment with and without this
          callbacks=[reduce_lr, early_stopping])  # Add early stopping to prevent overfitting

# Step 6: Load and Preprocess Test Data with Missing Image Handling
def load_test_data(test_dir):
    test_images = []
    test_ids = []
    missing_images = []
    for img_name in os.listdir(test_dir):
        img_path = os.path.join(test_dir, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (128, 128))
            test_images.append(img)
            test_ids.append(img_name.split('.')[0])
        else:
            print(f"Warning: Could not load image {img_name}")
            missing_images.append(img_name)

    print(f"Loaded {len(test_images)} images.")
    print(f"Missing images: {missing_images}")

    return np.array(test_images), test_ids, missing_images

# Load test data
test_images, test_ids, missing_images = load_test_data(test_dir)

# Normalize test data
test_images = test_images / 255.0

# Step 7: Make Predictions
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1) + 1  # Adjust to match original labels

# Ensure predicted_classes has 479 entries
if len(predicted_classes) < 479:
    print(f"Expected 479 images, but loaded {len(test_ids)}. Adding placeholder rows for missing images.")

    # Initialize predicted_classes if not already done
    predicted_classes = np.array(predicted_classes)

    # Add placeholder predictions (e.g., class 1) to match the expected number of rows
    for missing_img in missing_images:
        test_ids.append(missing_img.split('.')[0])
        predicted_classes = np.append(predicted_classes, [1])  # Default prediction as class 1

# Step 8: Create Submission File
submission = pd.DataFrame({
    'ID': test_ids,
    'Predictions': predicted_classes
})

# Save submission file
submission.to_csv('submission.csv', index=False)

# Step 9: Download Submission File
from google.colab import files
files.download('submission.csv')


Mounted at /content/drive
Loaded 2516 images.
Shape of images array: (2516, 128, 128, 3)
Shape of labels array: (2516,)
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/40


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


63/63 ━━━━━━━━━━━━━━━━━━━━ 327s 4s/step - accuracy: 0.2057 - loss: 2.0219 - val_accuracy: 0.0913 - val_loss: 1.7279 - learning_rate: 5.0000e-05
Epoch 2/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 250s 4s/step - accuracy: 0.2860 - loss: 1.7198 - val_accuracy: 0.1845 - val_loss: 1.6301 - learning_rate: 5.0000e-05
Epoch 3/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 251s 4s/step - accuracy: 0.2912 - loss: 1.6242 - val_accuracy: 0.0972 - val_loss: 1.6537 - learning_rate: 5.0000e-05
Epoch 4/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 256s 4s/step - accuracy: 0.3487 - loss: 1.5758 - val_accuracy: 0.1885 - val_loss: 1.6100 - learning_rate: 5.0000e-05
Epoch 5/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 246s 4s/step - accuracy: 0.3902 - loss: 1.4807 - val_accuracy: 0.1964 - val_loss: 1.7000 - learning_rate: 5.0000e-05
Epoch 6/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 260s 4s/step - accuracy: 0.3603 - loss: 1.4411 - val_accuracy: 0.1468 - val_loss: 1.7643 - learning_rate: 5.0000e-05
Epoch 7/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 260s 4s/step - accuracy: 0.4136 - loss: 1.3

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Import necessary libraries
from google.colab import drive
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

# Step 1: Mount Google Drive to access data
drive.mount('/content/drive')

# Define directories for training and test data
train_dir = '/content/drive/My Drive/Project 1 Data (1)/Project 1 Data/Train_Data'
test_dir = '/content/drive/My Drive/Project 1 Data (1)/Project 1 Data/Test_Data'

# Step 2: Load and Preprocess Training Data
IMG_SIZE = (224, 224)  # Increased image size
ALLOWED_EXTENSIONS = {'.jpg', '.jpeg', '.png'}

# Function to load training data
def load_data(train_dir):
    images = []
    labels = []
    label_map = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'S': 5}

    for folder in os.listdir(train_dir):
        folder_path = os.path.join(train_dir, folder)
        if os.path.isdir(folder_path) and folder in label_map:
            for img_name in os.listdir(folder_path):
                if not any(img_name.lower().endswith(ext) for ext in ALLOWED_EXTENSIONS):
                    continue
                img_path = os.path.join(folder_path, img_name)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, IMG_SIZE)
                    images.append(img)
                    labels.append(label_map[folder])
    return np.array(images), np.array(labels)

# Load training data
if os.path.exists(train_dir):
    images, labels = load_data(train_dir)
    print(f"Loaded {len(images)} images.")
    print(f"Shape of images array: {images.shape}")
    print(f"Shape of labels array: {labels.shape}")

# Step 3: Prepare Data for Training
labels = labels - 1  # Adjust labels to be 0-based

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Advanced Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    rescale=1./255,
    brightness_range=[0.8, 1.2],  # New augmentation for brightness
    channel_shift_range=0.1  # New augmentation for channel shifts
)
val_datagen = ImageDataGenerator(rescale=1./255)

# Step 4: Define the Model Using EfficientNetB1
base_model = EfficientNetB1(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Unfreeze all layers for fine-tuning
for layer in base_model.layers:
    layer.trainable = True

# Define the Sequential model
model = Sequential([
    base_model,
    Flatten(),
    Dense(1024, activation='relu', kernel_regularizer=l2(0.001)),  # Added L2 regularization
    Dropout(0.5),
    Dense(512, activation='relu', kernel_regularizer=l2(0.001)),  # Additional dense layer with L2 regularization
    Dropout(0.5),
    Dense(5, activation='softmax')  # Assuming 5 classes
])

# Compile the model with AdamW optimizer and a lower learning rate for fine-tuning
model.compile(optimizer=AdamW(learning_rate=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Step 5: Training the Model
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=1e-7)
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

# Train the model
batch_size = 32
model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
          validation_data=val_datagen.flow(X_val, y_val),
          epochs=25,  # Increased number of epochs for better convergence
          class_weight=class_weights_dict,
          callbacks=[reduce_lr])

# Step 6: Load and Preprocess Test Data with Missing Image Handling
def load_test_data(test_dir):
    test_images = []
    test_ids = []
    missing_images = []
    for img_name in os.listdir(test_dir):
        img_path = os.path.join(test_dir, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, IMG_SIZE)
            test_images.append(img)
            test_ids.append(img_name.split('.')[0])
        else:
            print(f"Warning: Could not load image {img_name}")
            missing_images.append(img_name)

    print(f"Loaded {len(test_images)} images.")
    print(f"Missing images: {missing_images}")

    return np.array(test_images), test_ids, missing_images

# Load test data
test_images, test_ids, missing_images = load_test_data(test_dir)

# Normalize test data
test_images = test_images / 255.0

# Step 7: Make Predictions
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1) + 1  # Adjust to match original labels

# Ensure predicted_classes has 479 entries
if len(predicted_classes) < 479:
    print(f"Expected 479 images, but loaded {len(test_ids)}. Adding placeholder rows for missing images.")

    # Initialize predicted_classes if not already done
    predicted_classes = np.array(predicted_classes)

    # Add placeholder predictions (e.g., class 1) to match the expected number of rows
    for missing_img in missing_images:
        test_ids.append(missing_img.split('.')[0])
        predicted_classes = np.append(predicted_classes, [1])  # Default prediction as class 1

# Step 8: Create Submission File
submission = pd.DataFrame({
    'ID': test_ids,
    'Predictions': predicted_classes
})

# Save submission file
submission.to_csv('submission.csv', index=False)

# Step 9: Download Submission File
from google.colab import files
files.download('submission.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded 2516 images.
Shape of images array: (2516, 224, 224, 3)
Shape of labels array: (2516,)
27018416/27018416 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/25


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


63/63 ━━━━━━━━━━━━━━━━━━━━ 1266s 19s/step - accuracy: 0.2624 - loss: 5.3559 - val_accuracy: 0.1310 - val_loss: 4.3728 - learning_rate: 1.0000e-05
Epoch 2/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 1098s 17s/step - accuracy: 0.2501 - loss: 4.9447 - val_accuracy: 0.1012 - val_loss: 4.5074 - learning_rate: 1.0000e-05
Epoch 3/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 1137s 18s/step - accuracy: 0.2245 - loss: 4.8522 - val_accuracy: 0.2857 - val_loss: 4.3705 - learning_rate: 1.0000e-05
Epoch 4/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 1169s 18s/step - accuracy: 0.2597 - loss: 4.6898 - val_accuracy: 0.2778 - val_loss: 4.3120 - learning_rate: 1.0000e-05
Epoch 5/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 1200s 19s/step - accuracy: 0.2606 - loss: 4.6235 - val_accuracy: 0.2798 - val_loss: 4.2813 - learning_rate: 1.0000e-05
Epoch 6/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 1086s 17s/step - accuracy: 0.2808 - loss: 4.5625 - val_accuracy: 0.2619 - val_loss: 4.2969 - learning_rate: 1.0000e-05
Epoch 7/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 1114s 18s/step - accuracy: 0.29